In [ ]:
%gui qt5

from nidaqmx import Task
from nidaqmx.constants import AcquisitionType, Edge
from nidaqmx.stream_readers import AnalogMultiChannelReader
from nidaqmx.stream_writers import AnalogMultiChannelWriter
import pyqtgraph as pg
import numpy as np


class ConfocalMicroscopy:
    def __init__(self, samp_rate=200000, amp=3, num_px=200):
        self.dev = "Dev1"
        self.ao_chx = "ao0"
        # fast
        self.ao_chy = "ao3" # slow
        self.ai_ch = "ai7"
        
        self.samp_rate = samp_rate # samples per channel per second
        self.amp = amp # scanning range in volt
        self.num_px = num_px # the number of pixels for both x and y (aspect ratio = 1)
        self.ao_range = 10 # the min/max value you expect to generate
        self.ai_range = 5 # the min/max value you expect to measure

        self.write_signal = self.waveform()
        self.total_px = self.num_px * self.num_px
        self.read_buffer = np.empty((1, self.total_px))
        self.bf_size = 3 # times bigger than one frame
        
        self.viewer = pg.ImageView()
        self.viewer.ui.roiBtn.hide()
        self.viewer.ui.menuBtn.hide()
        self.viewer.show()
        self.first_img = True
        
        self.start()

    def waveform(self):
        # triangular wave for the fast mirror
        line = np.linspace(-1, 1, self.num_px)
        x = np.tile(np.r_[line, line[::-1]], self.num_px // 2)
        if self.num_px % 2:
            x = np.r_[x, line]
        y = np.repeat(line, self.num_px)
        return np.stack([x, y]) * self.amp
    
    def set_tasks(self):
        self.write_task = Task()
        self.write_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_chx}", min_val=-self.ao_range, max_val=self.ao_range)
        self.write_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_chy}", min_val=-self.ao_range, max_val=self.ao_range)
        self.write_task.timing.cfg_samp_clk_timing(
            rate=self.samp_rate,
            source="OnboardClock",
            active_edge=Edge.RISING,
            sample_mode=AcquisitionType.CONTINUOUS,
        )
        self.write_task.out_stream.output_buf_size = 2 * self.total_px * self.bf_size
        self.write_task.register_every_n_samples_transferred_from_buffer_event(self.total_px, self.write_callback)
        self.writer = AnalogMultiChannelWriter(self.write_task.out_stream)
        # first write before starting the task
        for _ in range(self.bf_size):
            self.writer.write_many_sample(self.write_signal)

        self.read_task = Task()
        self.read_task.ai_channels.add_ai_voltage_chan(f"{self.dev}/{self.ai_ch}", min_val=-self.ai_range, max_val=self.ai_range)
        self.read_task.timing.cfg_samp_clk_timing(
            rate=self.samp_rate,
            source="OnboardClock",
            active_edge=Edge.RISING,
            sample_mode=AcquisitionType.CONTINUOUS,
        )
        self.read_task.in_stream.input_buf_size = self.total_px * self.bf_size
        self.read_task.triggers.start_trigger.cfg_dig_edge_start_trig(f"/{self.dev}/ao/StartTrigger", Edge.RISING)
        self.read_task.register_every_n_samples_acquired_into_buffer_event(self.total_px, self.read_callback)
        self.reader = AnalogMultiChannelReader(self.read_task.in_stream)

    def write_callback(self, task_handle, every_n_samples_event_type, number_of_samples, callback_data):
        self.writer.write_many_sample(self.write_signal, timeout=1)
        return 0
    
    def read_callback(self, task_handle, every_n_samples_event_type, number_of_samples, callback_data):
        self.reader.read_many_sample(self.read_buffer, number_of_samples, timeout=1)
        self.update(self.reconstruct_image())
        return 0

    def reconstruct_image(self):
        hardcoded_shift = -10
        img = self.read_buffer.copy()
        img = np.roll(img, hardcoded_shift).reshape(self.num_px, self.num_px)
        img[0::2, :] = img[0::2, ::-1]
        img = np.flipud(img)
        return img
        
    def update(self, img):
        self.viewer.setImage(
            img.T,
            autoLevels=self.first_img,
            autoHistogramRange=self.first_img,
        )
        self.first_img = False
        
    def start(self):
        self.set_tasks()
        self.read_task.start() # start read before write
        self.write_task.start()

In [ ]:
gui = ConfocalMicroscopy(samp_rate=50000, amp=6, num_px=300)
# fps = samp_rate / num_px**2

In [ ]:
# the tasks need to be closed before starting them again
gui.read_task.stop()
gui.read_task.close()
gui.write_task.stop()
gui.write_task.close()